In [1]:
##Importing packages we need##

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
from matplotlib import cm
from matplotlib.cm import get_cmap
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import metpy.calc as mpcalc
from metpy.units import units
from numpy import *
import xarray as xr
from netCDF4 import Dataset, num2date
import math
import pygrib
import cdsapi
import imageio
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import metpy as mp
import warnings
import glob
import dask
warnings.filterwarnings("ignore")

print("Done importing modules")

Done importing modules


In [2]:
##Declare time, level, lat/lon boundaries and loop through a 2-day period for Frontal Analysis##

#Area and Time#
colorado_area = [44, -112, 34, -99]  #some buffer on all sides
latN = 44
latS = 34
lonW = -112    #Must be in degrees E (The western hemisphere is captured between 180 and 360 degrees east)
lonE = -99
level = 850    #What level do we want to look at FLF on   

In [3]:
##Make a process data function for gifs##

def process_CF_data(ds_sfc):
    '''This function does a lot of calculations and outputs the timeseries with CAA and FLF.'''
    
    ##Load in the datasets and read in variables and cords##

    #This is for the surface level#
    dew_2m = ds_sfc.d2m.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    uwnd_10m = ds_sfc.u10.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    vwnd_10m = ds_sfc.v10.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    tmpk_2m = ds_sfc.t2m.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    mslp = ds_sfc.msl.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))
    sfc_p = ds_sfc.sp.metpy.sel(latitude=slice(latN,latS),longitude=slice(lonW,lonE))

    #Extract Coordinates#
    lats = ds_sfc.latitude.metpy.sel(latitude=slice(latN,latS))
    lons = ds_sfc.longitude.metpy.sel(longitude=slice(lonW,lonE))
    lons_2D, lats_2D = meshgrid(lons,lats)
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    tmpk_2m
    
    ##Apply a spatial smoother to the variables so that synoptic-scale signals can be more readily observed (25)##

    smoothing_var = 15

    #Surface variables#
    dew_2m = mpcalc.smooth_gaussian(dew_2m, smoothing_var)
    uwnd_10m = mpcalc.smooth_gaussian(uwnd_10m, smoothing_var)
    vwnd_10m = mpcalc.smooth_gaussian(vwnd_10m, smoothing_var)
    tmpk_2m = mpcalc.smooth_gaussian(tmpk_2m, smoothing_var)
    mslp = mpcalc.smooth_gaussian(mslp, smoothing_var)
    sfc_p = mpcalc.smooth_gaussian(sfc_p, smoothing_var)
    
    ##Calculate additional variables to plot##

    #Surface level variables#
    thetaE = mpcalc.equivalent_potential_temperature(sfc_p, tmpk_2m, dew_2m) 
    thetaE_adv = mpcalc.advection(thetaE, uwnd_10m, vwnd_10m)

    #Scale by a factor#
    thetaE_adv_s = thetaE_adv * 1e4

    #Select only CAA and WAA#
    thetaE_adv_s_CAA = thetaE_adv_s.where(thetaE_adv_s < 0, other=np.nan)
    thetaE_adv_s_WAA = thetaE_adv_s.where(thetaE_adv_s > 0, other=np.nan)

    thetaE_adv_s
    
    ##Calculate Front Locator Function (FLF) using ThetaE##

    def calculate_flf(thetaE, dx, dy):
        '''Make an FLF function to calculate it for all gridpoints at each timestep.'''

        #Step 1: Calculate the gradient of thetaE#
        d0_dy, d0_dx = mpcalc.gradient(thetaE, deltas=(dy, dx)) 

        #Step 2: Calculate the magnitude of the gradient#
        mag_grad_thetaE = np.sqrt((d0_dx**2) + (d0_dy**2))  # |grad(thetaE)|

        #Step 3a: Calculate the gradient of the magnitude#
        grad_step2_y, grad_step2_x = mpcalc.gradient(mag_grad_thetaE, deltas=(dy, dx))

        #Step 3b: Redo but for Qn component only#
        qx = grad_step2_x.copy()
        qy = grad_step2_y.copy()
        denominator = ((d0_dx) * (d0_dx)) + ((d0_dy) * (d0_dy))

        Qn_x = (((qx) * (d0_dx) * (d0_dx)) + ((qy) * (d0_dy) * (d0_dx))) / (denominator)  #i component
        Qn_y = (((qx) * (d0_dx) * (d0_dy)) + ((qy) * (d0_dy) * (d0_dy))) / (denominator)  #j component
        #Qn_low = Qn_x + Qn_y
        #Qn = Qn_low * 1e10

        #Step 4: Calculate the divergence of the gradient of the magnitude with Qn component#
        div_step4_Qn = mpcalc.divergence(Qn_x, Qn_y, dx=dx, dy=dy)
        div_step4_Qn_convert = div_step4_Qn * 1e14  #What we need

        return div_step4_Qn_convert

    #Apply the function across all timesteps now#
    FLF_data = xr.apply_ufunc(
        calculate_flf, 
        thetaE, 
        input_core_dims=[['latitude', 'longitude']],  #Core dimensions over which to apply the function
        output_core_dims=[['latitude', 'longitude']], #Core dimensions on the output
        vectorize=True,  #Automatically vectorize the computation if needed
        dask='parallelized',  #Use Dask for parallel computation if thetaE is a Dask-backed array
        kwargs={'dx': dx, 'dy': dy})  #Additional keyword arguments for the function
    
    return FLF_data, thetaE, lons, lats

In [4]:
##Import the Test Case##

ds_sfc = xr.open_dataset('CF_DATA/ERA5_01_22_2008.nc').metpy.parse_cf()
ds_sfc

FLFplot, thetaEplot, lons, lats = process_CF_data(ds_sfc)
FLFplot

<xarray.DataArray (time: 72, latitude: 41, longitude: 53)>
array([[[ 0.08343501,  0.07619061,  0.05702798, ...,  0.73683167,
          0.73248847,  0.60186624],
        [-0.05334626,  0.05718374,  0.09809669, ...,  0.60271881,
          0.52366179,  0.34679527],
        [-0.02902754,  0.12428114,  0.1185446 , ...,  0.40773101,
          0.32279085,  0.18647399],
        ...,
        [ 0.10053705,  0.12444766,  0.0426767 , ...,  0.37207714,
          0.43421659,  0.24676621],
        [ 0.30903255,  0.42149421,  0.37383313, ...,  0.29963963,
          0.40737843,  0.38759093],
        [ 0.5365028 ,  0.67124699,  0.57758712, ...,  0.17012918,
          0.51170699,  1.01250281]],

       [[ 0.06541397, -0.02746766, -0.22107067, ...,  0.78234144,
          0.99964807,  0.92949824],
        [-0.05003737, -0.0538626 , -0.13389813, ...,  0.59763702,
          0.75489139,  0.67570745],
        [-0.16218224, -0.07473139, -0.07054088, ...,  0.32042727,
          0.49691533,  0.52916865],
...
        [ 0.29740841,  0.18426795,  0.11169464, ...,  0.08105765,
          0.26016181,  0.48787755],
        [-0.07172471, -0.11081416, -0.13869006, ...,  0.11621469,
          0.19970959,  0.3791476 ],
        [ 0.09261709, -0.05441523, -0.14523297, ...,  0.13664074,
          0.14256243,  0.32215125]],

       [[ 0.71914989,  0.57367056, -0.17111707, ..., -0.07654368,
          0.14424806,  0.71316596],
        [ 0.83593179,  0.39707568, -0.36640782, ...,  0.02993999,
          0.18505066,  0.49862068],
        [ 0.13842548, -0.23624429, -0.7972563 , ...,  0.28055997,
          0.57917802,  0.94603666],
        ...,
        [ 0.01801597, -0.05563884, -0.06810455, ...,  0.08091522,
          0.25940209,  0.50189979],
        [ 0.05601936, -0.06960721, -0.14949597, ...,  0.11247332,
          0.19051261,  0.37308346],
        [ 0.04948808, -0.04788758, -0.1373203 , ...,  0.12914103,
          0.13259861,  0.31011435]]])
Coordinates:
  * longitude  (longitude) float32 -112.0 -111.8 -111.5 ... -99.5 -99.25 -99.0
  * latitude   (latitude) float32 44.0 43.75 43.5 43.25 ... 34.5 34.25 34.0
  * time       (time) datetime64[ns] 2008-01-20 ... 2008-01-22T23:00:00
    metpy_crs  object Projection: latitude_longitude

In [11]:
##Fix the date column## 

def refine_year(date):
    '''Make sure the years are correct.'''
    
    if date.year < 1950:
        return date + pd.DateOffset(years=100)  
    
    elif date.year > 2022:
        return date - pd.DateOffset(years=100)  
    
    return date

#Apply it to the data#
df = pd.read_csv('CF Cases_ManF_ UN.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y', errors='coerce').apply(refine_year)
df_2 = df[48:]
df_2

,Date,Max Value,Max Time
48,1986-11-01,NaN,NaN
49,1989-11-02,NaN,NaN
50,1994-11-09,NaN,NaN
51,2015-11-12,NaN,NaN
52,1956-11-15,NaN,NaN
53,1962-11-16,NaN,NaN
54,1981-11-27,NaN,NaN
55,1957-11-28,NaN,NaN
56,1977-11-28,NaN,NaN
57,2008-11-28,NaN,NaN


In [12]:
### Loop through each timestep and plot FLF##

#Process each case listed in the spreadsheet#
for index, row in df_2.iterrows():
    case_date = row['Date']
    #max_time = row['Max Time'] 
    
    #Assuming the dataset file name format is consistent#
    py_date = pd.to_datetime(case_date).to_pydatetime()
    formatted_date = py_date.strftime('%m_%d_%Y')
    dataset_path = f'CF_DATA/ERA5_{formatted_date}.nc' 
    
    #Import the test case#
    ds_sfc = xr.open_dataset(dataset_path).metpy.parse_cf()
    FLFplot, thetaEplot, lons, lats = process_CF_data(ds_sfc)
    
    #Case name based on date#
    case_name = str(FLFplot['time'][48].values)[0:10]
    
    #Create the main directory and subdirectory#
    main_dir = "GIFs_Unc"
    sub_dir = os.path.join(main_dir, case_name)
    os.makedirs(sub_dir, exist_ok=True)

    #Now loop through to make images and the gif#
    for i, time in enumerate(FLFplot['time']):

        #Setting up the plotting map and map projection#
        mapcrs = ccrs.LambertConformal(central_longitude=-55, central_latitude=45, standard_parallels=(33, 45)) 
        mapcrs = ccrs.PlateCarree()

        #Set up the projection of the data; if lat/lon then PlateCarree is what you want#
        datacrs = ccrs.PlateCarree()

        #Start the figure and create plot axes with proper projection#
        fig = plt.figure(1, figsize=(12, 10)) 
        ax = plt.subplot(111, projection=mapcrs) 
        ax.set_extent([-110, -100, 35, 43], ccrs.PlateCarree())  #-110, -100, 35, 43 | -110, -60, 20, 60

        #Add geopolitical boundaries for map reference#
        ax.add_feature(cfeature.LAND, facecolor="white") 
        countries = NaturalEarthFeature(category="cultural", scale="110m", facecolor="none", name="admin_0_boundary_lines_land") 
        ax.add_feature(countries, linewidth=0.5, edgecolor="black") 
        ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.5) 
        ax.coastlines('50m', linewidth=0.8)

        #Data to plot
        FLFs = FLFplot.sel(time=time)
        FLFs_th = FLFs.where(FLFs <= -0.1, other=np.nan)
        Theta_Es = thetaEplot.sel(time=time)
        #thetaE_advs = thetaE_adv_s_CAA.sel(time=time)

        #Set up contour and fill intervals#
        Qn_levels = arange(-3, 0.2, 0.2) 
        thetaE_levels = arange(250,400,4)
        #thetaE_adv_levels = arange(-15,1,1)
        #thetaE_adv_levels = arange(0,7.5,0.5)

        #Create the plot
        FLF_fconts = plt.contourf(lons, lats, FLFs_th, levels=Qn_levels, cmap='Blues_r', alpha=1, transform=ccrs.PlateCarree())
        #thetaE_adv_fconts = plt.contourf(lons, lats, thetaE_advs, levels=thetaE_adv_levels, cmap='Blues_r', alpha=1, transform=ccrs.PlateCarree())
        thetaE_conts = plt.contour(lons, lats, Theta_Es, levels=thetaE_levels, colors="red", alpha=0.7, linestyles='dashed',linewidths=2.0, transform=ccrs.PlateCarree())

        #Colorbar and contour labels#
        cb = fig.colorbar(FLF_fconts, orientation='vertical', pad=0.03, extendrect=True, aspect=25, shrink=0.8)
        cb.set_label('Scaled FLF (K/m^3 * 1e14)', size='x-large')
        #cb = fig.colorbar(thetaE_adv_fconts, orientation='vertical', pad=0.03, extendrect=True, aspect=25, shrink=0.8)
        #cb.set_label('ThetaE Advection (K/s * 1e4)', size='x-large')

        #Add a rectangle domain#
        rect_domain = patches.Rectangle(
            (-104.25, 38),  #(lon_min, lat_min)
            2,              #width (lon_max - lon_min)
            1.5,            #height (lat_max - lat_min)
            linewidth=2,
            edgecolor='black',
            facecolor='none'
        )
        ax.add_patch(rect_domain)

        #Set titles and labels
        ax.set_xticks(range(-110, -99, 1), crs=ccrs.PlateCarree())
        ax.set_yticks(range(35, 44, 1), crs=ccrs.PlateCarree())
        ax.set_xlabel('Longitude', fontsize=14)
        ax.set_ylabel('Latitude', fontsize=14)
        ax.set_title(f'FLF for Event: {str(time.values)}', fontsize=16) 

        #Save the current frame#
        frame_filename = os.path.join(sub_dir, f'frame_{i:04d}.png')
        plt.savefig(frame_filename)
        plt.close()

    #Specify the directory where your images are located#
    image_folder = sub_dir
    images = []

    #Gather the file names of the PNGs#
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.endswith('.png'):

            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))

    #Save the images as a GIF#
    gif_filename = os.path.join(main_dir, f'{case_name}.gif')
    imageio.mimsave(gif_filename, images, fps=5)  # Adjust fps to your liking
    
    print("Case Completed: ", case_name)
    

Case Completed:  1986-11-01
Case Completed:  1989-11-02
Case Completed:  1994-11-09
Case Completed:  2015-11-12
Case Completed:  1956-11-15
Case Completed:  1962-11-16
Case Completed:  1981-11-27
Case Completed:  1957-11-28
Case Completed:  1977-11-28
Case Completed:  2008-11-28
Case Completed:  1959-12-09
Case Completed:  2019-12-10
Case Completed:  1957-12-11
Case Completed:  1990-12-13
Case Completed:  1982-12-19
Case Completed:  1976-12-20
Case Completed:  2011-12-20
Case Completed:  2017-12-22
Case Completed:  1993-12-28
